# Librerías

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import os
import re
import unicodedata
from collections import Counter, defaultdict
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from xgboost import XGBRegressor


# Configuración Inicial

In [2]:
URL = "https://www.cnsf.gob.mx/EntidadesSupervisadas/InstitucionesSociedadesMutualistas/Paginas/Incendio.aspx"
CARPETA_DESCARGA = "data_cnsf"

In [3]:
dataframes_cnsf = {}
dataframes_limpios = []
descargas_exitosas = 0

# Obtención de datos

## Crear la carpeta de descarga

In [4]:
if not os.path.exists(CARPETA_DESCARGA):
    os.makedirs(CARPETA_DESCARGA)
    print(f"Carpeta '{CARPETA_DESCARGA}' creada para almacenar los archivos.")

## Descarga de archivos

In [5]:
print("\nIniciando descarga de archivos XLSX desde CNSF...")

try:
    # 2.1 Obtener el contenido de la página web
    response = requests.get(URL)
    response.raise_for_status()
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # 2.2 Encontrar enlaces que terminan en .xlsx o .xls
    enlaces_xlsx = soup.find_all('a', href=lambda href: href and (href.endswith('.xlsx') or href.endswith('.xls')))
    
    if not enlaces_xlsx:
        print("🚨 Advertencia: No se encontraron enlaces .xlsx/.xls. Verifique la URL.")
        
    
    for enlace in enlaces_xlsx:
        enlace_relativo = enlace['href']
        
        # 2.3 Reconstruir la URL completa
        if not enlace_relativo.startswith('http'):
            enlace_completo = "https://www.cnsf.gob.mx" + enlace_relativo
        else:
            enlace_completo = enlace_relativo

        nombre_archivo = enlace_completo.split('/')[-1]
        ruta_guardado = os.path.join(CARPETA_DESCARGA, nombre_archivo)
        
        # 2.4 Descargar el archivo si no existe localmente
        if not os.path.exists(ruta_guardado):
            print(f"Descargando: {nombre_archivo}...")
            archivo_response = requests.get(enlace_completo)
            archivo_response.raise_for_status()
            
            with open(ruta_guardado, 'wb') as f:
                f.write(archivo_response.content)
            descargas_exitosas += 1
        else:
            print(f"Omitido: {nombre_archivo} (Ya existe).")

except requests.exceptions.RequestException as e:
    print(f"🔴 Error al acceder a la URL o descargar archivos: {e}")


Iniciando descarga de archivos XLSX desde CNSF...
Omitido: 2024 Incendio Bases.xlsx (Ya existe).
Omitido: 2023 Incendio Bases.xlsx (Ya existe).
Omitido: 2022 Incendio Bases.xlsx (Ya existe).
Omitido: 2021 Incendio Bases.xlsx (Ya existe).
Omitido: 2020 Incendio Bases.xlsx (Ya existe).
Omitido: 2019 Incendio Bases.xlsx (Ya existe).
Omitido: 2018 Incendio Bases.xlsx (Ya existe).
Omitido: 2017 Incendio Bases.xlsx (Ya existe).
Omitido: 2016 Incendio Bases.xlsx (Ya existe).
Omitido: 2015 Incendio Bases.xlsx (Ya existe).
Omitido: 2014 Incendio Bases.xlsx (Ya existe).
Omitido: 2013 Incendio Bases.xlsx (Ya existe).
Omitido: 2012 Incendio Bases.xlsx (Ya existe).
Omitido: 2011 Incendio Bases.xlsx (Ya existe).
Omitido: 2010 Incendio Bases.xlsx (Ya existe).
Omitido: 2009 Incendio Bases.xlsx (Ya existe).
Omitido: 2008 Incendio Bases.xlsx (Ya existe).
Omitido: 2007 Incendio bases.xlsx (Ya existe).


## Carga de dataframes

### Emision

In [6]:
# Diccionario para guardar los dataframes crudos
dataframes_cnsf = {}

print(f"\nIniciando carga individual de archivos descargados en: {CARPETA_DESCARGA}...")

archivos_encontrados = os.listdir(CARPETA_DESCARGA)

for nombre_archivo in archivos_encontrados:
    if nombre_archivo.endswith('.xlsx') or nombre_archivo.endswith('.xls'):
        ruta_archivo = os.path.join(CARPETA_DESCARGA, nombre_archivo)
        
        # Intentar extraer el año (ej: '2007') del nombre del archivo para usarlo como clave
        match_ano = re.search(r'(\d{4})', nombre_archivo)
        
        if match_ano:
            clave_df = match_ano.group(1)
        else:
            clave_df = nombre_archivo.replace('.xlsx', '').replace('.xls', '')

        try:
            # INTENTO 1: Buscar hoja "Emision" (sin acento)
            # Nota: Quitamos los corchetes [] en sheet_name para obtener un DataFrame directo
            try:
                df = pd.read_excel(
                    ruta_archivo,
                    sheet_name="Emision",      
                    skiprows=0,            
                    header=1                
                )
            except ValueError:
                # INTENTO 2: Si falla, buscar hoja "Emisión" (con acento)
                print(f"   ℹ️ 'Emision' no encontrada en {clave_df}. Probando 'Emisión'...")
                df = pd.read_excel(
                    ruta_archivo,
                    sheet_name="Emisión",      
                    skiprows=0,            
                    header=1                
                )
            
            # Agregar columna de origen para trazabilidad (opcional pero recomendado)
            df['Origen_Archivo'] = nombre_archivo

            # Almacenar el DataFrame en el diccionario para manipulación individual
            dataframes_cnsf[clave_df] = df
            print(f"✅ Cargado: '{nombre_archivo}' en la clave '{clave_df}'. Filas: {len(df)}")

        except Exception as e:
            print(f"⚠️ ERROR al leer {nombre_archivo}: {e}. Revisar si existe la hoja 'Emision' o 'Emisión'.")


print(f"\n--- Resumen de Carga Individual ---")
print(f"Total de DataFrames cargados: {len(dataframes_cnsf)}")
print(f"Claves disponibles para limpieza individual: {list(dataframes_cnsf.keys())}")


Iniciando carga individual de archivos descargados en: data_cnsf...
   ℹ️ 'Emision' no encontrada en 2007. Probando 'Emisión'...
✅ Cargado: '2007 Incendio bases.xlsx' en la clave '2007'. Filas: 36444
✅ Cargado: '2008 Incendio Bases.xlsx' en la clave '2008'. Filas: 56759
✅ Cargado: '2009 Incendio Bases.xlsx' en la clave '2009'. Filas: 56758
✅ Cargado: '2010 Incendio Bases.xlsx' en la clave '2010'. Filas: 56758
✅ Cargado: '2011 Incendio Bases.xlsx' en la clave '2011'. Filas: 57937
✅ Cargado: '2012 Incendio Bases.xlsx' en la clave '2012'. Filas: 56758
✅ Cargado: '2013 Incendio Bases.xlsx' en la clave '2013'. Filas: 56758
✅ Cargado: '2014 Incendio Bases.xlsx' en la clave '2014'. Filas: 56758
✅ Cargado: '2015 Incendio Bases.xlsx' en la clave '2015'. Filas: 71394
✅ Cargado: '2016 Incendio Bases.xlsx' en la clave '2016'. Filas: 85693
✅ Cargado: '2017 Incendio Bases.xlsx' en la clave '2017'. Filas: 89165
✅ Cargado: '2018 Incendio Bases.xlsx' en la clave '2018'. Filas: 91239
✅ Cargado: '2019 I

In [7]:
# Asumiendo que 'dataframes_cnsf' es tu diccionario de DataFrames cargados
if dataframes_cnsf:
    print("--- 🔍 Nombres de Columnas por Archivo (Clave) ---")
    
    # Recorrer cada DataFrame en el diccionario
    for clave, df in dataframes_cnsf.items():
        print("--------------------------------------------------")
        print(f"🔑 Clave: {clave} (Origen: {df.get('Origen_Archivo', 'Desconocido')})")
        
        # Obtener los nombres de las columnas como una lista
        nombres_columnas = df.columns.tolist()
        
        print(f"Total de Columnas: {len(nombres_columnas)}")
        print("Nombres: ", nombres_columnas)
        print("--------------------------------------------------")
        
else:
    print("El diccionario 'dataframes_cnsf' está vacío. Asegúrate de que el proceso de carga se haya ejecutado correctamente.")

--- 🔍 Nombres de Columnas por Archivo (Clave) ---
--------------------------------------------------
🔑 Clave: 2007 (Origen: 0        2007 Incendio bases.xlsx
1        2007 Incendio bases.xlsx
2        2007 Incendio bases.xlsx
3        2007 Incendio bases.xlsx
4        2007 Incendio bases.xlsx
                   ...           
36439    2007 Incendio bases.xlsx
36440    2007 Incendio bases.xlsx
36441    2007 Incendio bases.xlsx
36442    2007 Incendio bases.xlsx
36443    2007 Incendio bases.xlsx
Name: Origen_Archivo, Length: 36444, dtype: object)
Total de Columnas: 15
Nombres:  ['MONEDA', 'FORMA DE VENTA', 'GIRO DE LA UBICACIÓN', 'ENTIDAD', 'NUMERO DE NIVELES', 'NÚMERO DE UBICACIONES', 'PRIMA EMITIDA', 'PRIMA RETENIDA', 'PRIMA DEVENGADA', 'COMISION DIRECTA', 'VALORES TOTALES EDIFICIO', 'VALORES TOTALES CONTENIDO', 'VALORES TOTALES EXISTENCIAS', ' ', 'Origen_Archivo']
--------------------------------------------------
--------------------------------------------------
🔑 Clave: 2008 (Origen

In [8]:
def estandarizar_columnas(df):
    """Limpia y estandariza los nombres de las columnas de un DataFrame."""
    # Crear un diccionario para mapear nombres antiguos a nombres nuevos
    column_mapping = {}
    
    for col in df.columns:
        # 1. Convertir a minúsculas
        new_col = str(col).lower()
        
        # 2. Reemplazar saltos de línea, espacios múltiples, y caracteres especiales por guion bajo
        new_col = new_col.replace('\n', ' ').strip()
        new_col = re.sub(r'[^a-z0-9]+', '_', new_col)
        new_col = new_col.strip('_')
        
        # 3. Aplicar mapeo de conceptos clave (ajusta estas reglas según tu análisis)
        if 'institucion' in new_col:
            new_col = 'nombre_institucion'
        elif 'prima' in new_col and 'emitida' in new_col:
            new_col = 'prima_emitida_neta'
        elif 'siniestralidad' in new_col:
            new_col = 'tasa_siniestralidad'
            
        column_mapping[col] = new_col
        
    return df.rename(columns=column_mapping)

In [9]:
dataframes_limpios = []

print("Iniciando estandarización y limpieza de DataFrames individuales...")

for clave, df in dataframes_cnsf.items():
    
    # 1. Copiar el DF para no modificar el original en el diccionario
    df_temp = df.copy()
    
    # 2. Aplicar la estandarización de columnas
    df_temp = estandarizar_columnas(df_temp)
    
    # ----------------------------------------------------------------------
    # 🎯 PUNTO DE INSERCIÓN CORRECTO: AGREGAR EL AÑO
    # ----------------------------------------------------------------------
    
    # 3. Agregar metadatos cruciales (la clave es el año extraído del nombre del archivo)
    df_temp['anio_origen'] = clave 
    
    # ----------------------------------------------------------------------
    
    # 4. Limpieza de filas de totales o vacías
    df_temp.dropna(how='all', inplace=True) 
    
    # 5. Si existe la columna, eliminar filas con 'TOTAL'
    if 'nombre_institucion' in df_temp.columns:
        df_temp = df_temp[~df_temp['nombre_institucion'].astype(str).str.contains('total', case=False, na=False)]
    
    dataframes_limpios.append(df_temp)
    print(f"✅ DF {clave} estandarizado y listo para concatenación.")


# CONCATENACIÓN FINAL
if dataframes_limpios:
    # Usamos pd.concat con join='outer' (comportamiento por defecto)
    # Esto une verticalmente los DataFrames por los nombres de columna estandarizados.
    # Si un DataFrame no tiene una columna, se rellena con NaN (Not a Number).
    df_consolidado = pd.concat(dataframes_limpios, ignore_index=True, axis=0)
    
    print("\n--- ✅ Consolidación Exitosa ---")
    print(f"Total de registros consolidados: {len(df_consolidado)}")
    print(f"Total de columnas ÚNICAS: {df_consolidado.shape[1]}")
    print("Primeras columnas consolidadas:", df_consolidado.columns.tolist()[:10])
    
    # df_consolidado.head()
    
else:
    print("\n❌ No hay DataFrames limpios para consolidar.")

Iniciando estandarización y limpieza de DataFrames individuales...
✅ DF 2007 estandarizado y listo para concatenación.
✅ DF 2008 estandarizado y listo para concatenación.
✅ DF 2009 estandarizado y listo para concatenación.
✅ DF 2010 estandarizado y listo para concatenación.
✅ DF 2011 estandarizado y listo para concatenación.
✅ DF 2012 estandarizado y listo para concatenación.
✅ DF 2013 estandarizado y listo para concatenación.
✅ DF 2014 estandarizado y listo para concatenación.
✅ DF 2015 estandarizado y listo para concatenación.
✅ DF 2016 estandarizado y listo para concatenación.
✅ DF 2017 estandarizado y listo para concatenación.
✅ DF 2018 estandarizado y listo para concatenación.
✅ DF 2019 estandarizado y listo para concatenación.
✅ DF 2020 estandarizado y listo para concatenación.
✅ DF 2021 estandarizado y listo para concatenación.
✅ DF 2022 estandarizado y listo para concatenación.
✅ DF 2023 estandarizado y listo para concatenación.
✅ DF 2024 estandarizado y listo para concatenació

In [10]:
df_EMI=df_consolidado.copy()

In [11]:
df_EMI.columns

Index(['moneda', 'forma_de_venta', 'giro_de_la_ubicaci_n', 'entidad',
       'numero_de_niveles', 'n_mero_de_ubicaciones', 'prima_emitida_neta',
       'prima_retenida', 'prima_devengada', 'comision_directa',
       'valores_totales_edificio', 'valores_totales_contenido',
       'valores_totales_existencias', '', 'origen_archivo', 'anio_origen',
       'n_mero_de_niveles', 'tipo_primer_riesgo', 'comisi_n_directa',
       'valores_totales_p_rdidas_consecuenciales',
       'l_mite_m_ximo_de_responsabilidad', 'subtipo_de_seguro',
       'prima_devengada_en_el_ejercicio', 'prima_devengada_acumulada'],
      dtype='object')

In [12]:
df_EMI.head()

,moneda,forma_de_venta,giro_de_la_ubicaci_n,entidad,numero_de_niveles,n_mero_de_ubicaciones,prima_emitida_neta,prima_retenida,prima_devengada,comision_directa,...,origen_archivo,anio_origen,n_mero_de_niveles,tipo_primer_riesgo,comisi_n_directa,valores_totales_p_rdidas_consecuenciales,l_mite_m_ximo_de_responsabilidad,subtipo_de_seguro,prima_devengada_en_el_ejercicio,prima_devengada_acumulada
0,Extranjera,Agentes Persona Física,Cultivo de granos y semillas oleaginosas,DISTRITO FEDERAL,1.0,3.0,133966.0,92657.0,0.0,7128.0,...,2007 Incendio bases.xlsx,2007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Extranjera,Agentes Persona Física,Cultivo de granos y semillas oleaginosas,DISTRITO FEDERAL,2.0,2.0,72670.0,51718.0,25062.0,2659.0,...,2007 Incendio bases.xlsx,2007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Extranjera,Agentes Persona Física,Cultivo de granos y semillas oleaginosas,GUANAJUATO,2.0,3.0,30148.0,6030.0,14802.0,7537.0,...,2007 Incendio bases.xlsx,2007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Extranjera,Agentes Persona Física,Cultivo de granos y semillas oleaginosas,NAYARIT,1.0,1.0,34810.0,24774.0,0.0,1274.0,...,2007 Incendio bases.xlsx,2007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Extranjera,Agentes Persona Física,Cultivo de granos y semillas oleaginosas,NUEVO LEON,1.0,1.0,172321.0,122638.0,0.0,6305.0,...,2007 Incendio bases.xlsx,2007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Suma Asegurada

In [13]:
print(f"\nIniciando carga individual de {descargas_exitosas} archivos descargados...")

archivos_encontrados = os.listdir(CARPETA_DESCARGA)

for nombre_archivo in archivos_encontrados:
    if nombre_archivo.endswith('.xlsx') or nombre_archivo.endswith('.xls'):
        ruta_archivo = os.path.join(CARPETA_DESCARGA, nombre_archivo)
        
        # Intentar extraer el año (ej: '2007') del nombre del archivo para usarlo como clave
        match_ano = re.search(r'(\d{4})', nombre_archivo)
        
        if match_ano:
            clave_df = match_ano.group(1)
        else:
            clave_df = nombre_archivo.replace('.xlsx', '').replace('.xls', '')

        try:
            # Parámetros iniciales de lectura (pueden necesitar ajuste manual por archivo)
            df = pd.read_excel(
                ruta_archivo,
                sheet_name="Suma Asegurada",          
                skiprows=0,            
                header=1               
            )
            
            # Almacenar el DataFrame en el diccionario para manipulación individual
            dataframes_cnsf[clave_df] = df
            print(f"✅ Cargado: '{nombre_archivo}' en la clave '{clave_df}'. Filas: {len(df)}")

        except Exception as e:
            print(f"⚠️ ERROR al leer {nombre_archivo}: {e}. Revisar estructura.")


print(f"\n--- Resumen de Carga Individual ---")
print(f"Total de DataFrames cargados: {len(dataframes_cnsf)}")
print(f"Claves disponibles para limpieza individual: {list(dataframes_cnsf.keys())}")


Iniciando carga individual de 0 archivos descargados...
✅ Cargado: '2007 Incendio bases.xlsx' en la clave '2007'. Filas: 87002
✅ Cargado: '2008 Incendio Bases.xlsx' en la clave '2008'. Filas: 131995
✅ Cargado: '2009 Incendio Bases.xlsx' en la clave '2009'. Filas: 129346
✅ Cargado: '2010 Incendio Bases.xlsx' en la clave '2010'. Filas: 140184
✅ Cargado: '2011 Incendio Bases.xlsx' en la clave '2011'. Filas: 142480
✅ Cargado: '2012 Incendio Bases.xlsx' en la clave '2012'. Filas: 167681
✅ Cargado: '2013 Incendio Bases.xlsx' en la clave '2013'. Filas: 156139
✅ Cargado: '2014 Incendio Bases.xlsx' en la clave '2014'. Filas: 186317
✅ Cargado: '2015 Incendio Bases.xlsx' en la clave '2015'. Filas: 183761
✅ Cargado: '2016 Incendio Bases.xlsx' en la clave '2016'. Filas: 215546
✅ Cargado: '2017 Incendio Bases.xlsx' en la clave '2017'. Filas: 227531
✅ Cargado: '2018 Incendio Bases.xlsx' en la clave '2018'. Filas: 227261
✅ Cargado: '2019 Incendio Bases.xlsx' en la clave '2019'. Filas: 226479
✅ Cargad

In [14]:

if dataframes_cnsf:
    print("--- 🔍 Nombres de Columnas por Archivo (Clave) ---")
    
    # Recorrer cada DataFrame en el diccionario
    for clave, df in dataframes_cnsf.items():
        print("--------------------------------------------------")
        print(f"🔑 Clave: {clave} (Origen: {df.get('Origen_Archivo', 'Desconocido')})")
        
        # Obtener los nombres de las columnas como una lista
        nombres_columnas = df.columns.tolist()
        
        print(f"Total de Columnas: {len(nombres_columnas)}")
        print("Nombres: ", nombres_columnas)
        print("--------------------------------------------------")
        
else:
    print("El diccionario 'dataframes_cnsf' está vacío. Asegúrate de que el proceso de carga se haya ejecutado correctamente.")

--- 🔍 Nombres de Columnas por Archivo (Clave) ---
--------------------------------------------------
🔑 Clave: 2007 (Origen: Desconocido)
Total de Columnas: 6
Nombres:  ['GIRO', 'ENTIDAD', 'TIPO DE BIEN', 'COBERTURA', 'NÚMERO DE UBICACIONES', 'SUMA ASEGURADA']
--------------------------------------------------
--------------------------------------------------
🔑 Clave: 2008 (Origen: Desconocido)
Total de Columnas: 8
Nombres:  ['GIRO', 'ENTIDAD', 'TIPO PRIMER RIESGO', 'TIPO DE BIEN', 'COBERTURA ', 'NÚMERO DE UBICACIONES', 'SUMA ASEGURADA', 'SUMA ASEGURADA EXPUESTA']
--------------------------------------------------
--------------------------------------------------
🔑 Clave: 2009 (Origen: Desconocido)
Total de Columnas: 8
Nombres:  ['GIRO', 'ENTIDAD', 'TIPO PRIMER RIESGO', 'TIPO DE BIEN', 'COBERTURA ', 'NÚMERO DE UBICACIONES', 'SUMA ASEGURADA', 'SUMA ASEGURADA EXPUESTA']
--------------------------------------------------
--------------------------------------------------
🔑 Clave: 2010 (O

In [15]:
def estandarizar_columnas(df):
    """Limpia y estandariza los nombres de las columnas de un DataFrame."""
    # Crear un diccionario para mapear nombres antiguos a nombres nuevos
    column_mapping = {}
    
    for col in df.columns:
        # 1. Convertir a minúsculas
        new_col = str(col).lower()
        
        # 2. Reemplazar saltos de línea, espacios múltiples, y caracteres especiales por guion bajo
        new_col = new_col.replace('\n', ' ').strip()
        new_col = re.sub(r'[^a-z0-9]+', '_', new_col)
        new_col = new_col.strip('_')
        
        # 3. Aplicar mapeo de conceptos clave (ajusta estas reglas según tu análisis)
        if 'institucion' in new_col:
            new_col = 'nombre_institucion'
        elif 'prima' in new_col and 'emitida' in new_col:
            new_col = 'prima_emitida_neta'
        elif 'siniestralidad' in new_col:
            new_col = 'tasa_siniestralidad'
            
        column_mapping[col] = new_col
        
    return df.rename(columns=column_mapping)

In [16]:
dataframes_limpios = []

print("Iniciando estandarización y limpieza de DataFrames individuales...")

for clave, df in dataframes_cnsf.items():
    
    # 1. Copiar el DF para no modificar el original en el diccionario
    df_temp = df.copy()
    
    # 2. Aplicar la estandarización de columnas
    df_temp = estandarizar_columnas(df_temp)
    
    # ----------------------------------------------------------------------
    # 🎯 PUNTO DE INSERCIÓN CORRECTO: AGREGAR EL AÑO
    # ----------------------------------------------------------------------
    
    # 3. Agregar metadatos cruciales (la clave es el año extraído del nombre del archivo)
    df_temp['anio_origen'] = clave 
    
    # ----------------------------------------------------------------------
    
    # 4. Limpieza de filas de totales o vacías
    df_temp.dropna(how='all', inplace=True) 
    
    # 5. Si existe la columna, eliminar filas con 'TOTAL'
    if 'nombre_institucion' in df_temp.columns:
        df_temp = df_temp[~df_temp['nombre_institucion'].astype(str).str.contains('total', case=False, na=False)]
    
    dataframes_limpios.append(df_temp)
    print(f"✅ DF {clave} estandarizado y listo para concatenación.")


# CONCATENACIÓN FINAL
if dataframes_limpios:
    # Usamos pd.concat con join='outer' (comportamiento por defecto)
    # Esto une verticalmente los DataFrames por los nombres de columna estandarizados.
    # Si un DataFrame no tiene una columna, se rellena con NaN (Not a Number).
    df_consolidado = pd.concat(dataframes_limpios, ignore_index=True, axis=0)
    
    print("\n--- ✅ Consolidación Exitosa ---")
    print(f"Total de registros consolidados: {len(df_consolidado)}")
    print(f"Total de columnas ÚNICAS: {df_consolidado.shape[1]}")
    print("Primeras columnas consolidadas:", df_consolidado.columns.tolist()[:10])
    
    # df_consolidado.head()
    
else:
    print("\n❌ No hay DataFrames limpios para consolidar.")

Iniciando estandarización y limpieza de DataFrames individuales...
✅ DF 2007 estandarizado y listo para concatenación.
✅ DF 2008 estandarizado y listo para concatenación.
✅ DF 2009 estandarizado y listo para concatenación.
✅ DF 2010 estandarizado y listo para concatenación.
✅ DF 2011 estandarizado y listo para concatenación.
✅ DF 2012 estandarizado y listo para concatenación.
✅ DF 2013 estandarizado y listo para concatenación.
✅ DF 2014 estandarizado y listo para concatenación.
✅ DF 2015 estandarizado y listo para concatenación.
✅ DF 2016 estandarizado y listo para concatenación.
✅ DF 2017 estandarizado y listo para concatenación.
✅ DF 2018 estandarizado y listo para concatenación.
✅ DF 2019 estandarizado y listo para concatenación.
✅ DF 2020 estandarizado y listo para concatenación.
✅ DF 2021 estandarizado y listo para concatenación.
✅ DF 2022 estandarizado y listo para concatenación.
✅ DF 2023 estandarizado y listo para concatenación.
✅ DF 2024 estandarizado y listo para concatenació

In [17]:
df_SA=df_consolidado.copy()

In [18]:
df_SA.head()

,giro,entidad,tipo_de_bien,cobertura,n_mero_de_ubicaciones,suma_asegurada,anio_origen,tipo_primer_riesgo,suma_asegurada_expuesta,subtipo_de_seguro
0,Acabado y recubrimiento de textiles,AGUASCALIENTES,Contenidos,Remoción de Escombros,1,700000,2007,NaN,NaN,NaN
1,Acabado y recubrimiento de textiles,AGUASCALIENTES,Contenidos y Edificio,"Incendio, rayo y explosión",5,9500000,2007,NaN,NaN,NaN
2,Acabado y recubrimiento de textiles,AGUASCALIENTES,Contenidos y Edificio,Remoción de Escombros,3,1091570,2007,NaN,NaN,NaN
3,Acabado y recubrimiento de textiles,AGUASCALIENTES,Contenidos,"Incendio, rayo y explosión",2,8000000,2007,NaN,NaN,NaN
4,Acabado y recubrimiento de textiles,BAJA CALIFORNIA,Otra combinación,Extensión de cubierta (sin inundación),2,865615,2007,NaN,NaN,NaN


### Siniestros

In [19]:
print(f"\nIniciando carga individual de {descargas_exitosas} archivos descargados...")

archivos_encontrados = os.listdir(CARPETA_DESCARGA)

for nombre_archivo in archivos_encontrados:
    if nombre_archivo.endswith('.xlsx') or nombre_archivo.endswith('.xls'):
        ruta_archivo = os.path.join(CARPETA_DESCARGA, nombre_archivo)
        
        # Intentar extraer el año (ej: '2007') del nombre del archivo para usarlo como clave
        match_ano = re.search(r'(\d{4})', nombre_archivo)
        
        if match_ano:
            clave_df = match_ano.group(1)
        else:
            clave_df = nombre_archivo.replace('.xlsx', '').replace('.xls', '')

        try:
            # Parámetros iniciales de lectura (pueden necesitar ajuste manual por archivo)
            df = pd.read_excel(
                ruta_archivo,
                sheet_name="Siniestros",          
                skiprows=0,            
                header=1               
            )
            
            # Almacenar el DataFrame en el diccionario para manipulación individual
            dataframes_cnsf[clave_df] = df
            print(f"✅ Cargado: '{nombre_archivo}' en la clave '{clave_df}'. Filas: {len(df)}")

        except Exception as e:
            print(f"⚠️ ERROR al leer {nombre_archivo}: {e}. Revisar estructura.")


print(f"\n--- Resumen de Carga Individual ---")
print(f"Total de DataFrames cargados: {len(dataframes_cnsf)}")
print(f"Claves disponibles para limpieza individual: {list(dataframes_cnsf.keys())}")


Iniciando carga individual de 0 archivos descargados...
✅ Cargado: '2007 Incendio bases.xlsx' en la clave '2007'. Filas: 7680
✅ Cargado: '2008 Incendio Bases.xlsx' en la clave '2008'. Filas: 9128
✅ Cargado: '2009 Incendio Bases.xlsx' en la clave '2009'. Filas: 8864
✅ Cargado: '2010 Incendio Bases.xlsx' en la clave '2010'. Filas: 10256
✅ Cargado: '2011 Incendio Bases.xlsx' en la clave '2011'. Filas: 10503
✅ Cargado: '2012 Incendio Bases.xlsx' en la clave '2012'. Filas: 9236
✅ Cargado: '2013 Incendio Bases.xlsx' en la clave '2013'. Filas: 11237
✅ Cargado: '2014 Incendio Bases.xlsx' en la clave '2014'. Filas: 11015
✅ Cargado: '2015 Incendio Bases.xlsx' en la clave '2015'. Filas: 12176
✅ Cargado: '2016 Incendio Bases.xlsx' en la clave '2016'. Filas: 13483
✅ Cargado: '2017 Incendio Bases.xlsx' en la clave '2017'. Filas: 14091
✅ Cargado: '2018 Incendio Bases.xlsx' en la clave '2018'. Filas: 14956
✅ Cargado: '2019 Incendio Bases.xlsx' en la clave '2019'. Filas: 15291
✅ Cargado: '2020 Incendi

In [20]:
if dataframes_cnsf:
    print("--- 🔍 Nombres de Columnas por Archivo (Clave) ---")
    
    # Recorrer cada DataFrame en el diccionario
    for clave, df in dataframes_cnsf.items():
        print("--------------------------------------------------")
        print(f"🔑 Clave: {clave} (Origen: {df.get('Origen_Archivo', 'Desconocido')})")
        
        # Obtener los nombres de las columnas como una lista
        nombres_columnas = df.columns.tolist()
        
        print(f"Total de Columnas: {len(nombres_columnas)}")
        print("Nombres: ", nombres_columnas)
        print("--------------------------------------------------")
        
else:
    print("El diccionario 'dataframes_cnsf' está vacío. Asegúrate de que el proceso de carga se haya ejecutado correctamente.")

--- 🔍 Nombres de Columnas por Archivo (Clave) ---
--------------------------------------------------
🔑 Clave: 2007 (Origen: Desconocido)
Total de Columnas: 16
Nombres:  ['MONEDA', 'FORMA DE VENTA', 'GIRO DE LA UBICACIÓN', 'NUMERO DE NIVELES', 'ENTIDAD', 'TIPO DE BIEN', 'COBERTURA ', 'CAUSA DEL SINIESTRO', 'NUMERO DE SINIESTROS', 'MONTO DE SINIESTRO ', 'GASTO DE AJUSTE', 'SALVAMENTO', 'MONTO PAGADO', 'MONTO DE DEDUCIBLE', 'MONTO COASEGURO', ' ']
--------------------------------------------------
--------------------------------------------------
🔑 Clave: 2008 (Origen: Desconocido)
Total de Columnas: 17
Nombres:  ['MONEDA', 'FORMA DE VENTA', 'GIRO LA UBICACIÓN', 'NÚMERO DE NIVELES', 'TIPO PRIMER RIESGO', 'ENTIDAD', 'TIPO DE BIEN', 'COBERTURA', 'CAUSA DEL SINIESTRO', 'NÚMERO DE SINIESTROS', 'MONTO DE SINIESTRO', 'GASTO DE AJUSTE', 'SALVAMENTO', 'MONTO PAGADO', 'MONTO DE DEDUCIBLE', 'MONTO COASEGURO', ' ']
--------------------------------------------------
---------------------------------

In [21]:
def estandarizar_columnas(df):
    """Limpia y estandariza los nombres de las columnas de un DataFrame."""
    # Crear un diccionario para mapear nombres antiguos a nombres nuevos
    column_mapping = {}
    
    for col in df.columns:
        # 1. Convertir a minúsculas
        new_col = str(col).lower()
        
        # 2. Reemplazar saltos de línea, espacios múltiples, y caracteres especiales por guion bajo
        new_col = new_col.replace('\n', ' ').strip()
        new_col = re.sub(r'[^a-z0-9]+', '_', new_col)
        new_col = new_col.strip('_')
        
        # 3. Aplicar mapeo de conceptos clave (ajusta estas reglas según tu análisis)
        if 'institucion' in new_col:
            new_col = 'nombre_institucion'
        elif 'prima' in new_col and 'emitida' in new_col:
            new_col = 'prima_emitida_neta'
        elif 'siniestralidad' in new_col:
            new_col = 'tasa_siniestralidad'
            
        column_mapping[col] = new_col
        
    return df.rename(columns=column_mapping)

In [22]:
dataframes_limpios = []

print("Iniciando estandarización y limpieza de DataFrames individuales...")

for clave, df in dataframes_cnsf.items():
    
    # 1. Copiar el DF para no modificar el original en el diccionario
    df_temp = df.copy()
    
    # 2. Aplicar la estandarización de columnas
    df_temp = estandarizar_columnas(df_temp)
    
    # ----------------------------------------------------------------------
    # 🎯 PUNTO DE INSERCIÓN CORRECTO: AGREGAR EL AÑO
    # ----------------------------------------------------------------------
    
    # 3. Agregar metadatos cruciales (la clave es el año extraído del nombre del archivo)
    df_temp['anio_origen'] = clave 
    
    # ----------------------------------------------------------------------
    
    # 4. Limpieza de filas de totales o vacías
    df_temp.dropna(how='all', inplace=True) 
    
    # 5. Si existe la columna, eliminar filas con 'TOTAL'
    if 'nombre_institucion' in df_temp.columns:
        df_temp = df_temp[~df_temp['nombre_institucion'].astype(str).str.contains('total', case=False, na=False)]
    
    dataframes_limpios.append(df_temp)
    print(f"✅ DF {clave} estandarizado y listo para concatenación.")


# CONCATENACIÓN FINAL
if dataframes_limpios:
    # Usamos pd.concat con join='outer' (comportamiento por defecto)
    # Esto une verticalmente los DataFrames por los nombres de columna estandarizados.
    # Si un DataFrame no tiene una columna, se rellena con NaN (Not a Number).
    df_consolidado = pd.concat(dataframes_limpios, ignore_index=True, axis=0)
    
    print("\n--- ✅ Consolidación Exitosa ---")
    print(f"Total de registros consolidados: {len(df_consolidado)}")
    print(f"Total de columnas ÚNICAS: {df_consolidado.shape[1]}")
    print("Primeras columnas consolidadas:", df_consolidado.columns.tolist()[:10])
    
    # df_consolidado.head()
    
else:
    print("\n❌ No hay DataFrames limpios para consolidar.")

Iniciando estandarización y limpieza de DataFrames individuales...
✅ DF 2007 estandarizado y listo para concatenación.
✅ DF 2008 estandarizado y listo para concatenación.
✅ DF 2009 estandarizado y listo para concatenación.
✅ DF 2010 estandarizado y listo para concatenación.
✅ DF 2011 estandarizado y listo para concatenación.
✅ DF 2012 estandarizado y listo para concatenación.
✅ DF 2013 estandarizado y listo para concatenación.
✅ DF 2014 estandarizado y listo para concatenación.
✅ DF 2015 estandarizado y listo para concatenación.
✅ DF 2016 estandarizado y listo para concatenación.
✅ DF 2017 estandarizado y listo para concatenación.
✅ DF 2018 estandarizado y listo para concatenación.
✅ DF 2019 estandarizado y listo para concatenación.
✅ DF 2020 estandarizado y listo para concatenación.
✅ DF 2021 estandarizado y listo para concatenación.
✅ DF 2022 estandarizado y listo para concatenación.
✅ DF 2023 estandarizado y listo para concatenación.
✅ DF 2024 estandarizado y listo para concatenació

In [23]:
df_SIN=df_consolidado.copy()

In [24]:
df_SIN.head()

,moneda,forma_de_venta,giro_de_la_ubicaci_n,numero_de_niveles,entidad,tipo_de_bien,cobertura,causa_del_siniestro,numero_de_siniestros,monto_de_siniestro,...,monto_coaseguro,,anio_origen,giro_la_ubicaci_n,n_mero_de_niveles,tipo_primer_riesgo,n_mero_de_siniestros,subtipo_de_seguro,recuperacion_de_terceros,recuperacion_de_reaseguro
0,Extranjera,Agentes Persona Física,Extracción de petróleo y gas,15.0,TAMAULIPAS,Existencias / Bienes Bajo Convenio Expreso,Gastos Extraordinarios,Explosión,1.0,5606890.0,...,0.0,,2007,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Extranjera,Agentes Persona Física,Suministro de gas por ductos al consumidor final,1.0,BAJA CALIFORNIA,Contenidos,"Naves Aéreas, Vehículos y Humo",Caída De Avión,1.0,0.0,...,0.0,,2007,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Extranjera,Agentes Persona Física,Otra actividad o servicio relacionada con la e...,2.0,JALISCO,Contenidos,"Incendio, rayo y explosión",Auto Ignición,2.0,20000.0,...,0.0,,2007,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Extranjera,Agentes Persona Física,Molienda de granos y de semillas oleaginosas,1.0,DISTRITO FEDERAL,Edificio,"Naves Aéreas, Vehículos y Humo",Otra causa,1.0,494.0,...,0.0,,2007,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Extranjera,Agentes Persona Física,"Matanza, empacado y procesamiento de carne de ...",1.0,BAJA CALIFORNIA,Contenidos,Todo Riesgo,Impacto De Vehículos,2.0,0.0,...,0.0,,2007,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
df_EMI['giro'] = df_EMI['giro_de_la_ubicaci_n']

In [26]:
df_SIN['giro'] = df_SIN['giro_la_ubicaci_n'].fillna(df_SIN['giro_de_la_ubicaci_n'])
df_SIN['no_niveles'] = df_SIN['numero_de_niveles'].fillna(df_SIN['n_mero_de_niveles'])
df_SIN['no_siniestros'] = df_SIN['numero_de_siniestros'].fillna(df_SIN['n_mero_de_siniestros'])


### Checkpoint datos (guardar datos unificados)

In [27]:
#df_EMI.to_csv("EMI_07_24.csv")

In [28]:
#df_SA.to_csv("SA_07_24.csv")

In [29]:
#df_SIN.to_csv("SIN_07_24.csv")

### Carga datos unificados (opcional)

In [30]:
#df_EMI=pd.read_csv("EMI_07_24.csv")

In [31]:
#df_SA=pd.read_csv("SA_07_24.csv")

In [32]:
#df_SIN=pd.read_csv("SIN_07_24.csv")

# EDA

## Exploracion de datos

In [33]:
df_EMI.head()

,moneda,forma_de_venta,giro_de_la_ubicaci_n,entidad,numero_de_niveles,n_mero_de_ubicaciones,prima_emitida_neta,prima_retenida,prima_devengada,comision_directa,...,anio_origen,n_mero_de_niveles,tipo_primer_riesgo,comisi_n_directa,valores_totales_p_rdidas_consecuenciales,l_mite_m_ximo_de_responsabilidad,subtipo_de_seguro,prima_devengada_en_el_ejercicio,prima_devengada_acumulada,giro
0,Extranjera,Agentes Persona Física,Cultivo de granos y semillas oleaginosas,DISTRITO FEDERAL,1.0,3.0,133966.0,92657.0,0.0,7128.0,...,2007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cultivo de granos y semillas oleaginosas
1,Extranjera,Agentes Persona Física,Cultivo de granos y semillas oleaginosas,DISTRITO FEDERAL,2.0,2.0,72670.0,51718.0,25062.0,2659.0,...,2007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cultivo de granos y semillas oleaginosas
2,Extranjera,Agentes Persona Física,Cultivo de granos y semillas oleaginosas,GUANAJUATO,2.0,3.0,30148.0,6030.0,14802.0,7537.0,...,2007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cultivo de granos y semillas oleaginosas
3,Extranjera,Agentes Persona Física,Cultivo de granos y semillas oleaginosas,NAYARIT,1.0,1.0,34810.0,24774.0,0.0,1274.0,...,2007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cultivo de granos y semillas oleaginosas
4,Extranjera,Agentes Persona Física,Cultivo de granos y semillas oleaginosas,NUEVO LEON,1.0,1.0,172321.0,122638.0,0.0,6305.0,...,2007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cultivo de granos y semillas oleaginosas


In [34]:
df_SA.head()

,giro,entidad,tipo_de_bien,cobertura,n_mero_de_ubicaciones,suma_asegurada,anio_origen,tipo_primer_riesgo,suma_asegurada_expuesta,subtipo_de_seguro
0,Acabado y recubrimiento de textiles,AGUASCALIENTES,Contenidos,Remoción de Escombros,1,700000,2007,NaN,NaN,NaN
1,Acabado y recubrimiento de textiles,AGUASCALIENTES,Contenidos y Edificio,"Incendio, rayo y explosión",5,9500000,2007,NaN,NaN,NaN
2,Acabado y recubrimiento de textiles,AGUASCALIENTES,Contenidos y Edificio,Remoción de Escombros,3,1091570,2007,NaN,NaN,NaN
3,Acabado y recubrimiento de textiles,AGUASCALIENTES,Contenidos,"Incendio, rayo y explosión",2,8000000,2007,NaN,NaN,NaN
4,Acabado y recubrimiento de textiles,BAJA CALIFORNIA,Otra combinación,Extensión de cubierta (sin inundación),2,865615,2007,NaN,NaN,NaN


In [35]:
df_SIN.head()

,moneda,forma_de_venta,giro_de_la_ubicaci_n,numero_de_niveles,entidad,tipo_de_bien,cobertura,causa_del_siniestro,numero_de_siniestros,monto_de_siniestro,...,giro_la_ubicaci_n,n_mero_de_niveles,tipo_primer_riesgo,n_mero_de_siniestros,subtipo_de_seguro,recuperacion_de_terceros,recuperacion_de_reaseguro,giro,no_niveles,no_siniestros
0,Extranjera,Agentes Persona Física,Extracción de petróleo y gas,15.0,TAMAULIPAS,Existencias / Bienes Bajo Convenio Expreso,Gastos Extraordinarios,Explosión,1.0,5606890.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Extracción de petróleo y gas,15.0,1.0
1,Extranjera,Agentes Persona Física,Suministro de gas por ductos al consumidor final,1.0,BAJA CALIFORNIA,Contenidos,"Naves Aéreas, Vehículos y Humo",Caída De Avión,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Suministro de gas por ductos al consumidor final,1.0,1.0
2,Extranjera,Agentes Persona Física,Otra actividad o servicio relacionada con la e...,2.0,JALISCO,Contenidos,"Incendio, rayo y explosión",Auto Ignición,2.0,20000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Otra actividad o servicio relacionada con la e...,2.0,2.0
3,Extranjera,Agentes Persona Física,Molienda de granos y de semillas oleaginosas,1.0,DISTRITO FEDERAL,Edificio,"Naves Aéreas, Vehículos y Humo",Otra causa,1.0,494.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Molienda de granos y de semillas oleaginosas,1.0,1.0
4,Extranjera,Agentes Persona Física,"Matanza, empacado y procesamiento de carne de ...",1.0,BAJA CALIFORNIA,Contenidos,Todo Riesgo,Impacto De Vehículos,2.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Matanza, empacado y procesamiento de carne de ...",1.0,2.0


In [36]:
df_EMI = df_EMI.loc[:, ~df_EMI.columns.str.contains('^Unnamed')]
df_SA = df_SA.loc[:, ~df_SA.columns.str.contains('^Unnamed')]
df_SIN = df_SIN.loc[:, ~df_SIN.columns.str.contains('^Unnamed')]

## Tipado de datos

In [37]:
df_EMI['anio_origen']=df_EMI['anio_origen'].astype('str')
df_SA['anio_origen']=df_SA['anio_origen'].astype('str')
df_SIN['anio_origen']=df_SIN['anio_origen'].astype('str')

In [38]:
giros_EMI=df_EMI['giro']
giros_SA =df_SA['giro']
giros_SIN= df_SIN['giro']

## Estandarización de catálogos

### Catálogos

In [39]:
def normalizar_texto(s):
    if pd.isna(s):
        return s
    s = str(s).strip().lower()
    s = ''.join(
        c for c in unicodedata.normalize('NFKD', s)
        if not unicodedata.combining(c)
    )
    return s


In [40]:
def construir_catalogo(*series):
    conteo_norm = Counter()
    originales_por_norm = defaultdict(Counter)

    for ser in series:
        for val in ser.dropna():
            g_norm = normalizar_texto(val)
            conteo_norm[g_norm] += 1
            originales_por_norm[g_norm][val] += 1

    catalogo = {}
    for g_norm, counter_orig in originales_por_norm.items():
        mas_comunes = counter_orig.most_common()
        max_count = mas_comunes[0][1]
        candidatos = [s for s, c in mas_comunes if c == max_count]
        canonico = min(candidatos, key=len)
        catalogo[g_norm] = canonico

    return catalogo

In [41]:
catalogo_giros_unificado = construir_catalogo(
    df_EMI['giro'],
    df_SA['giro'],
    df_SIN['giro']
)



In [42]:
catalogo_entidades_unificado = construir_catalogo(
    df_EMI['entidad'],
    df_SA['entidad'],
    df_SIN['entidad']
)


### Variables Estandarizadas

#### Giro

In [43]:

for df in (df_EMI, df_SA, df_SIN):
    df['giro_norm'] = df['giro'].apply(normalizar_texto)
    df['giro_unificado'] = df['giro_norm'].map(catalogo_giros_unificado)

In [44]:
for nombre, df in [('EMI', df_EMI), ('SA', df_SA), ('SIN', df_SIN)]:
    sin_mapeo = df[df['giro_unificado'].isna()]['giro'].unique()
    print(f'[{nombre}] giros sin mapeo ({len(sin_mapeo)}):')
    print(sin_mapeo[:20])  # muestra solo los primeros 20
    print('---')

[EMI] giros sin mapeo (1):
[nan]
---
[SA] giros sin mapeo (0):
[]
---
[SIN] giros sin mapeo (1):
[nan]
---


In [45]:
catalogo_df = pd.DataFrame(
    list(catalogo_giros_unificado.items()),
    columns=['giro_norm', 'giro_unificado']
)

In [46]:
#catalogo_df.to_csv('cat_giros_unificados.csv')

#### Entidad

In [47]:
for df in (df_EMI, df_SA, df_SIN):
    df['entidad_norm'] = df['entidad'].apply(normalizar_texto)
    df['entidad_unificado'] = df['entidad_norm'].map(catalogo_entidades_unificado)

In [48]:
catalogo_ent = pd.DataFrame(
    list(catalogo_giros_unificado.items()),
    columns=['giro_norm', 'giro_unificado']
)

In [49]:
catalogo_ent.to_csv('cat_entidades_unificados.csv')

In [50]:
for nombre, df in [('EMI', df_EMI), ('SA', df_SA), ('SIN', df_SIN)]:
    sin_mapeo = df[df['giro_unificado'].isna()]['giro'].unique()
    print(f'[{nombre}] giros sin mapeo ({len(sin_mapeo)}):')
    print(sin_mapeo[:20])  # muestra solo los primeros 20
    print('---')

[EMI] giros sin mapeo (1):
[nan]
---
[SA] giros sin mapeo (0):
[]
---
[SIN] giros sin mapeo (1):
[nan]
---


In [102]:
# Mapeo faltante a mano

In [104]:
mapeo_manual_entidad = {
    "MEXICO": "Estado de México",
    "DISTRITO FEDERAL": "Ciudad de México",
    "NUEVO LEON": "Nuevo Leon",
    "EN EL EXTRANJERO": "Extranjero"
}



### Sector

In [51]:
catalogo = catalogo_df 
catalogo['giro_norm'] = catalogo['giro_norm'].apply(normalizar_texto)

In [52]:
sector_map = {}

# ---- AGROPECUARIO Y MINERÍA ----
for g in [
 'acuicultura animal','caza y captura','cultivo de frutales y nueces',
 'cultivo de granos y semillas oleaginosas','cultivo de hortalizas',
 'cultivo en invernaderos y viveros y floricultura',
 'explotacion avicola','explotacion de bovinos','explotacion de porcinos',
 'explotacion de ovinos y caprinos','mineria de carbon mineral',
 'mineria de minerales metalicos','mineria de minerales no metalicos',
 'extraccion de petroleo y gas','pesca','silvicultura','tala de arboles',
 'servicios relacionados con la ganaderia',
 'servicios relacionados con la agricultura',
 'servicios relacionados con la mineria'
]:
    sector_map[g] = 'Agropecuario y minería'

# ---- ENERGÍA Y AGUA ----
for g in catalogo['giro_norm']:
    if 'energia electrica' in g or 'agua' in g or 'gas por ductos' in g:
        sector_map[g] = 'Energía y agua'

# ---- CONSTRUCCIÓN ----
for g in catalogo['giro_norm']:
    if 'construccion' in g or 'cimentaciones' in g or 'edificacion' in g:
        sector_map[g] = 'Construcción'

# ---- MANUFACTURA ----
for g in catalogo['giro_norm']:
    if g.startswith('fabricacion') or 'industria' in g:
        sector_map[g] = 'Manufactura'

# ---- COMERCIO ----
for g in catalogo['giro_norm']:
    if 'comercio al por mayor' in g:
        sector_map[g] = 'Comercio al por mayor'
    elif 'comercio al por menor' in g:
        sector_map[g] = 'Comercio al por menor'

# ---- TRANSPORTE Y LOGÍSTICA ----
for g in catalogo['giro_norm']:
    if 'transporte' in g or 'mensajeria' in g or 'almacenamiento' in g:
        sector_map[g] = 'Transporte y logística'

# ---- SERVICIOS FINANCIEROS E INMOBILIARIOS ----
for g in catalogo['giro_norm']:
    if 'banca' in g or 'seguros' in g or 'inmobili' in g:
        sector_map[g] = 'Servicios financieros e inmobiliarios'

# ---- SERVICIOS PROFESIONALES Y EDUCACIÓN ----
for g in catalogo['giro_norm']:
    if 'consultoria' in g or 'escuela' in g or 'consultorios' in g:
        sector_map[g] = 'Servicios profesionales y educación'

# ---- GOBIERNO ----
for g in catalogo['giro_norm']:
    if 'administracion publica' in g or 'justicia' in g or 'seguridad nacional' in g:
        sector_map[g] = 'Gobierno'

# ---- VIVIENDA ----
for g in ['casa habitacion','departamento']:
    sector_map[g] = 'Vivienda'



In [53]:
def agregar_sector(df, catalogo, sector_map):
    df['giro_norm'] = df['giro'].apply(normalizar_texto)

    df = df.merge(
        catalogo[['giro_norm', 'giro_unificado']],
        on='giro_norm',
        how='left'
    )

    df['sector'] = df['giro_norm'].map(sector_map)

    df['sector'] = df['sector'].fillna('Otros')

    return df

In [54]:
df_EMI = agregar_sector(df_EMI, catalogo, sector_map)
df_SA = agregar_sector(df_SA,  catalogo, sector_map)
df_SIN = agregar_sector(df_SIN, catalogo, sector_map)

In [55]:
no_asignados = df_SIN[df_SIN['sector'].isna()]['giro'].unique()
print("Giros sin sector:", no_asignados)


Giros sin sector: []


In [56]:
df_SA.columns

Index(['giro', 'entidad', 'tipo_de_bien', 'cobertura', 'n_mero_de_ubicaciones',
       'suma_asegurada', 'anio_origen', 'tipo_primer_riesgo',
       'suma_asegurada_expuesta', 'subtipo_de_seguro', 'giro_norm',
       'giro_unificado_x', 'entidad_norm', 'entidad_unificado',
       'giro_unificado_y', 'sector'],
      dtype='object')

In [57]:
df_EMI['giro']=df_EMI['giro_unificado_x']
df_SA['giro']=df_SA['giro_unificado_x']
df_SIN['giro']=df_SIN['giro_unificado_x']

In [58]:
df_EMI['entidad']=df_EMI['entidad_unificado']
df_SA['entidad']=df_SA['entidad_unificado']
df_SIN['entidad']=df_SIN['entidad_unificado']

## Merge Emisión, Suma Asegurada y Siniestralidad

In [59]:
EMI_cols =['entidad','sector','giro' ,'anio_origen', 'prima_emitida_neta',
       'prima_retenida', 'prima_devengada'
          ]

SA_cols =[ 'entidad','sector','giro', 'anio_origen', 'suma_asegurada'
        ]

SIN_cols =['entidad','sector','giro','anio_origen', 'monto_de_siniestro', 'gasto_de_ajuste', 
           'salvamento', 'monto_pagado', 'monto_de_deducible', 'monto_coaseguro',
           'n_mero_de_siniestros', 'recuperacion_de_terceros', 
           'recuperacion_de_reaseguro' 
]

### Agregación de datos

In [60]:
cols_agrupar = ['giro','sector', 'entidad', 'anio_origen']

In [61]:
EMI_cols_sum = ['prima_emitida_neta', 'prima_retenida', 'prima_devengada']
SA_cols_sum = ['suma_asegurada']
SIN_cols_sum =['monto_de_siniestro', 'gasto_de_ajuste', 'salvamento', 'monto_pagado', 
               'monto_de_deducible', 'monto_coaseguro', 'n_mero_de_siniestros', 
               'recuperacion_de_terceros', 'recuperacion_de_reaseguro']

In [62]:
agg_dict_emi = {col: 'sum' for col in EMI_cols_sum}
df_EMI_agg = df_EMI[cols_agrupar + EMI_cols_sum].groupby(cols_agrupar).agg(agg_dict_emi).reset_index()

In [63]:
agg_dict_sa = {col: 'sum' for col in SA_cols_sum}
df_SA_agg = df_SA[cols_agrupar + SA_cols_sum].groupby(cols_agrupar).agg(agg_dict_sa).reset_index()

In [64]:
agg_dict_sin = {col: 'sum' for col in SIN_cols_sum}
df_SIN_agg = df_SIN[cols_agrupar + SIN_cols_sum].groupby(cols_agrupar).agg(agg_dict_sin).reset_index()

In [65]:
df_SIN_agg.shape

(37048, 13)

### Merge

In [66]:
df_SIN_IND = pd.merge(
    df_EMI_agg,
    df_SIN_agg,
    on=['giro','sector', 'entidad', 'anio_origen'],
    how='inner'  
)

In [67]:
df_QT = pd.merge(
    df_EMI_agg,
    df_SA_agg,
    on=['giro','sector', 'entidad', 'anio_origen'], 
    how='inner'  
)

In [68]:
df_TOT = pd.merge(
    df_SIN_IND,
    df_SA_agg,
    on=['giro','sector', 'entidad', 'anio_origen'], 
    how='inner'  
)

## Selección de variables

In [69]:
df_TOT.shape

(33876, 17)

In [70]:
df_TOT.isna().sum()

giro                         0
sector                       0
entidad                      0
anio_origen                  0
prima_emitida_neta           0
prima_retenida               0
prima_devengada              0
monto_de_siniestro           0
gasto_de_ajuste              0
salvamento                   0
monto_pagado                 0
monto_de_deducible           0
monto_coaseguro              0
n_mero_de_siniestros         0
recuperacion_de_terceros     0
recuperacion_de_reaseguro    0
suma_asegurada               0
dtype: int64

In [71]:
df_TOT.head()

,giro,sector,entidad,anio_origen,prima_emitida_neta,prima_retenida,prima_devengada,monto_de_siniestro,gasto_de_ajuste,salvamento,monto_pagado,monto_de_deducible,monto_coaseguro,n_mero_de_siniestros,recuperacion_de_terceros,recuperacion_de_reaseguro,suma_asegurada
0,Acabado y recubrimiento de textiles,Otros,Baja California,2013,6304.86,6304.86,13012.85,2450577.64,0.0,0.0,0.00,0.0,0.0,2.0,0.0,0.0,86340040
1,Acabado y recubrimiento de textiles,Otros,Baja California,2014,29308.53,0.00,14176.25,-6683137.95,0.0,0.0,0.00,0.0,0.0,5.0,0.0,0.0,86613935
2,Acabado y recubrimiento de textiles,Otros,Coahuila,2017,0.00,0.00,50158.60,55209.90,21622.0,0.0,27604.95,0.0,0.0,1.0,0.0,0.0,235954800
3,Acabado y recubrimiento de textiles,Otros,Colima,2017,0.00,-1971.35,23246.87,6475000.00,243431.0,0.0,0.00,0.0,0.0,1.0,0.0,0.0,43000000
4,Acabado y recubrimiento de textiles,Otros,DISTRITO FEDERAL,2007,2283792.00,1838107.00,2233248.00,375285.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,5066871198


## Tratamientos de nulos

In [72]:
df_proc=df_TOT.copy()

In [73]:
#Catalogar variables
cat_cols = ['giro', 'sector','entidad', 'anio_origen'
]

num_cols = ['prima_emitida_neta',
       'prima_retenida', 'prima_devengada', 'monto_de_siniestro',
       'gasto_de_ajuste', 'salvamento', 'monto_pagado', 'monto_de_deducible',
       'monto_coaseguro', 'n_mero_de_siniestros', 'recuperacion_de_terceros',
       'recuperacion_de_reaseguro', 'suma_asegurada'
]




In [74]:
df_proc[num_cols]=df_proc[num_cols].fillna(0)
df_proc[cat_cols]=df_proc[cat_cols].fillna('NA')

In [75]:
df_proc

,giro,sector,entidad,anio_origen,prima_emitida_neta,prima_retenida,prima_devengada,monto_de_siniestro,gasto_de_ajuste,salvamento,monto_pagado,monto_de_deducible,monto_coaseguro,n_mero_de_siniestros,recuperacion_de_terceros,recuperacion_de_reaseguro,suma_asegurada
0,Acabado y recubrimiento de textiles,Otros,Baja California,2013,6304.86,6304.86,13012.85,2450577.64,0.0,0.0,0.00,0.0,0.0,2.0,0.0,0.0,86340040
1,Acabado y recubrimiento de textiles,Otros,Baja California,2014,29308.53,0.00,14176.25,-6683137.95,0.0,0.0,0.00,0.0,0.0,5.0,0.0,0.0,86613935
2,Acabado y recubrimiento de textiles,Otros,Coahuila,2017,0.00,0.00,50158.60,55209.90,21622.0,0.0,27604.95,0.0,0.0,1.0,0.0,0.0,235954800
3,Acabado y recubrimiento de textiles,Otros,Colima,2017,0.00,-1971.35,23246.87,6475000.00,243431.0,0.0,0.00,0.0,0.0,1.0,0.0,0.0,43000000
4,Acabado y recubrimiento de textiles,Otros,DISTRITO FEDERAL,2007,2283792.00,1838107.00,2233248.00,375285.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,5066871198
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33871,Órganos legislativos,Otros,Tabasco,2008,43470.00,22134.00,34889.00,568.00,509.0,0.0,568.00,526.0,0.0,1.0,0.0,0.0,14031474
33872,Órganos legislativos,Otros,Veracruz,2007,7324.00,4327.00,7324.00,5000.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,927194056
33873,Órganos legislativos,Otros,Veracruz,2008,9422.00,5512.00,7560.00,-5000.00,0.0,0.0,0.00,0.0,0.0,1.0,0.0,0.0,3602566
33874,Órganos legislativos,Otros,Yucatán,2007,2172.00,1284.00,2172.00,0.00,3110.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,275029216


### Checkpoint datos (tablas para indicadores)

In [76]:
#df_proc.to_csv('datos_agregados_estandarizados.csv')

# Ingeniería de Variables

In [105]:
df_proc["entidad"] = df_proc["entidad"].replace(mapeo_manual_entidad)

In [106]:
df_proc.columns

Index(['giro', 'sector', 'entidad', 'año', 'prima_emitida_neta',
       'prima_retenida', 'prima_devengada', 'monto_de_siniestro',
       'gasto_de_ajuste', 'salvamento', 'monto_pagado', 'monto_de_deducible',
       'monto_coaseguro', 'n_mero_de_siniestros', 'recuperacion_de_terceros',
       'recuperacion_de_reaseguro', 'suma_asegurada', 'cuota_millar',
       'sin_index', 'siniestro_neto', 'net_sin_index'],
      dtype='object')

In [107]:
df_proc['cuota_millar'] = np.where(
    df_proc['suma_asegurada'] != 0,
    df_proc['prima_emitida_neta']*1000 / df_proc['suma_asegurada'],
    0
)

In [108]:
df_proc['sin_index'] = np.where(
    df_proc['prima_devengada'] != 0,
    df_proc['monto_de_siniestro'] / df_proc['prima_devengada'],
    0
)

In [109]:
df_proc['siniestro_neto'] = df_proc['monto_de_siniestro']+ df_proc['gasto_de_ajuste'] - df_proc['salvamento'] - df_proc['recuperacion_de_terceros'] - df_proc['recuperacion_de_reaseguro']
    

In [110]:
df_proc['net_sin_index'] = np.where(
    df_proc['prima_devengada'] != 0,
    df_proc['siniestro_neto'] / df_proc['prima_devengada'],
    0
)

In [111]:
df_proc.shape

(28987, 21)

In [112]:
df_proc.describe()

,año,prima_emitida_neta,prima_retenida,prima_devengada,monto_de_siniestro,gasto_de_ajuste,salvamento,monto_pagado,monto_de_deducible,monto_coaseguro,n_mero_de_siniestros,recuperacion_de_terceros,recuperacion_de_reaseguro,suma_asegurada,cuota_millar,sin_index,siniestro_neto,net_sin_index
count,28987.000000,2.898700e+04,2.898700e+04,2.898700e+04,2.898700e+04,2.898700e+04,2.898700e+04,2.898700e+04,2.898700e+04,2.898700e+04,28987.000000,2.898700e+04,2.898700e+04,2.898700e+04,28987.000000,2.898700e+04,2.898700e+04,2.898700e+04
mean,2016.677752,7.246256e+06,1.774495e+06,3.776335e+06,5.429889e+06,1.500757e+05,2.747997e+04,3.206536e+06,1.709843e+05,1.117937e+04,16.548798,4.963273e+03,1.096105e+06,9.782699e+10,0.412887,5.805201e+01,4.451416e+06,6.109033e+01
std,5.319084,7.995985e+07,9.571903e+06,4.486813e+07,5.848292e+07,1.055913e+06,6.846226e+05,2.697336e+07,3.364695e+06,3.925917e+05,117.284263,2.131711e+05,2.874458e+07,1.161320e+12,11.925734,6.103802e+03,5.076717e+07,6.190607e+03
min,2007.000000,-4.255194e+07,-1.072980e+07,-1.874632e+06,-7.073775e+08,-5.907677e+07,-3.858959e+06,-9.463393e+07,-6.329080e+05,0.000000e+00,0.000000,-2.503849e+06,-6.410628e+08,1.227740e+05,-14.011605,0.000000e+00,-1.992667e+08,0.000000e+00
25%,2012.000000,2.094791e+05,7.394603e+04,0.000000e+00,0.000000e+00,1.022560e+03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000,0.000000e+00,0.000000e+00,1.957369e+09,0.047009,0.000000e+00,4.700345e+03,0.000000e+00
50%,2018.000000,8.926637e+05,3.290452e+05,2.003192e+05,6.622000e+04,5.217710e+03,0.000000e+00,8.580000e+03,0.000000e+00,0.000000e+00,2.000000,0.000000e+00,0.000000e+00,9.269724e+09,0.103078,4.981646e-03,6.877400e+04,2.477457e-02
75%,2021.000000,3.416364e+06,1.127570e+06,1.460586e+06,7.000000e+05,3.601795e+04,0.000000e+00,3.161636e+05,0.000000e+00,0.000000e+00,6.000000,0.000000e+00,0.000000e+00,3.942818e+10,0.216275,2.951457e-01,6.268979e+05,3.663994e-01
max,2024.000000,7.972052e+09,8.477808e+08,3.450697e+09,4.847828e+09,5.742478e+07,9.142421e+07,1.475583e+09,2.473860e+08,3.992877e+07,7334.000000,2.500000e+07,3.629442e+09,1.028387e+14,1343.466000,1.024962e+06,4.847830e+09,1.038809e+06


In [113]:
df_proc=df_proc[df_proc['sin_index']>=0]

In [114]:
df_proc=df_proc[df_proc['net_sin_index']>=0]

In [115]:
df_TOT.shape

(33876, 17)

In [116]:
df_proc.shape

(28987, 21)

### Checkpoint datos indicadores

In [117]:
df_proc = df_proc.rename(columns={'anio_origen': 'año'})

In [118]:
df_proc['año']=df_proc['año'].astype(int)

In [119]:
df_proc.to_csv('datos_indicadores_estandarizados.csv')

# Entrenamiento del modelo

In [120]:
df_proc=pd.read_csv('datos_indicadores_estandarizados.csv')

In [121]:
df_proc = df_proc.loc[:, ~df_proc.columns.str.contains('^Unnamed')]

In [122]:
df = df_proc.copy()

In [123]:
df.columns

Index(['giro', 'sector', 'entidad', 'año', 'prima_emitida_neta',
       'prima_retenida', 'prima_devengada', 'monto_de_siniestro',
       'gasto_de_ajuste', 'salvamento', 'monto_pagado', 'monto_de_deducible',
       'monto_coaseguro', 'n_mero_de_siniestros', 'recuperacion_de_terceros',
       'recuperacion_de_reaseguro', 'suma_asegurada', 'cuota_millar',
       'sin_index', 'siniestro_neto', 'net_sin_index'],
      dtype='object')

## Selección de variables

In [124]:
def prediccion_siniestralidad(df_proc, giro_usuario, entidad_usuario, min_obs=3):
    # 1. Columnas con lag
    lag_features = [
        'prima_emitida_neta', 'prima_retenida', 'prima_devengada',
        'monto_de_siniestro', 'gasto_de_ajuste', 'salvamento',
        'monto_pagado', 'monto_de_deducible', 'monto_coaseguro',
        'n_mero_de_siniestros', 'recuperacion_de_terceros',
        'recuperacion_de_reaseguro', 'suma_asegurada', 'cuota_millar',
        'sin_index', 'siniestro_neto', 'net_sin_index'
    ]
    
    # --- helper: entrena y predice para un nivel dado (giro o sector) ---
    def _ajustar_y_predecir(df_base, cat_col, valor_cat, nivel_desc):
        """
        df_base: dataframe YA filtrado al nivel deseado (giro+entidad o sector+entidad)
        cat_col: 'giro' o 'sector'
        valor_cat: giro_usuario o sector_usuario
        nivel_desc: solo para mensajes ('giro' o 'sector')
        """
        df = df_base.copy()
        df = df.sort_values(['entidad', 'año'])

        # Asegurar numérico en columnas base
        cols_num = [c for c in lag_features if c in df.columns]
        df[cols_num] = df[cols_num].apply(pd.to_numeric, errors='coerce')

        # Crear lags dentro de este subset
        for col in lag_features:
            if col in df.columns:
                df[f'{col}_lag1'] = df.groupby([cat_col, 'entidad'])[col].shift(1)

        # Asegurar numérico en lags
        lag1_cols = [f'{c}_lag1' for c in lag_features if f'{c}_lag1' in df.columns]
        if lag1_cols:
            df[lag1_cols] = df[lag1_cols].apply(pd.to_numeric, errors='coerce')

        # Limpiar target y valores raros
        df = df[df['net_sin_index'].notna()]
        df = df.replace([np.inf, -np.inf], np.nan)
        df_model = df.dropna(subset=lag1_cols + ['net_sin_index'])

        if df_model.shape[0] < min_obs:
            raise ValueError(
                f"No hay suficientes datos limpios a nivel {nivel_desc} para {cat_col}={valor_cat}, entidad={entidad_usuario}"
            )

        # OneHot de categoría y entidad
        ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
        encoded = ohe.fit_transform(df_model[[cat_col, 'entidad']])
        ohe_cols = list(ohe.get_feature_names_out())
        encoded_df = pd.DataFrame(encoded, columns=ohe_cols, index=df_model.index)

        # Unir lags + OHE
        df_model_enc = pd.concat([df_model, encoded_df], axis=1)

        # Features: solo lags + OHE
        lag1_cols = [c for c in df_model_enc.columns if c.endswith('_lag1')]
        feature_cols = lag1_cols + ohe_cols

        X = df_model_enc[feature_cols]
        y = df_model_enc['net_sin_index']

        if y.isna().any():
            raise ValueError(f"y (net_sin_index) aún tiene NaNs a nivel {nivel_desc}.")

        # Modelo
        model = XGBRegressor(
            n_estimators=600,
            learning_rate=0.05,
            max_depth=5,
            subsample=0.9,
            colsample_bytree=0.9
        )
        model.fit(X, y)

        # Base para predicción: última observación en el tiempo
        base = df.sort_values('año').iloc[-1].copy()
        last_year = int(base['año'])

        predicciones = {}
        current = base.copy()

        for step in [1, 2]:  # año+1 y año+2
            año_futuro = last_year + step

            # OHE para el valor actual (giro o sector) + entidad_usuario
            encoded_row = ohe.transform([[valor_cat, entidad_usuario]])
            encoded_row_df = pd.DataFrame(encoded_row, columns=ohe_cols)

            # Tomamos SOLO los lags actuales
            row_lags = current[lag1_cols].to_frame().T.reset_index(drop=True)

            # Unimos lags + OHE
            row = pd.concat(
                [row_lags, encoded_row_df.reset_index(drop=True)],
                axis=1
            )

            # Asegurar orden de columnas igual que X
            row = row.reindex(columns=feature_cols, fill_value=0).astype(float)

            pred = model.predict(row)[0]
            predicciones[año_futuro] = float(pred)   # <-- aquí usamos el año real como llave

            # Actualizar lag de net_sin_index para el siguiente paso
            if 'net_sin_index_lag1' in current.index:
                current['net_sin_index_lag1'] = pred

        return predicciones

    # =====================================================
    #  NIVEL 1: modelo por GIRO + ENTIDAD
    # =====================================================
    df_ge = df_proc[(df_proc['giro'] == giro_usuario) &
                    (df_proc['entidad'] == entidad_usuario)].copy()

    preds_giro = None

    if df_ge.shape[0] >= min_obs:
        try:
            preds_giro = _ajustar_y_predecir(
                df_ge, cat_col='giro',
                valor_cat=giro_usuario,
                nivel_desc='giro'
            )
            # Si todas las predicciones son ~0, consideramos que no sirve y hacemos fallback
            if not all(abs(v) < 1e-9 for v in preds_giro.values()):
                return preds_giro  # ✅ usamos el modelo por giro
        except ValueError:
            preds_giro = None  # seguimos al nivel sector

    # =====================================================
    #  NIVEL 2: fallback por SECTOR + ENTIDAD
    # =====================================================
    subset_giro = df_proc[df_proc['giro'] == giro_usuario]
    if subset_giro.empty or subset_giro['sector'].isna().all():
        raise ValueError(f"No se encontró sector asociado al giro={giro_usuario}")

    sector_usuario = subset_giro['sector'].mode().iloc[0]

    df_se = df_proc[(df_proc['sector'] == sector_usuario) &
                    (df_proc['entidad'] == entidad_usuario)].copy()

    if df_se.shape[0] < min_obs:
        raise ValueError(
            f"No hay suficientes datos ni a nivel giro ni a nivel sector para "
            f"giro={giro_usuario}, entidad={entidad_usuario}, sector={sector_usuario}"
        )

    preds_sector = _ajustar_y_predecir(
        df_se, cat_col='sector',
        valor_cat=sector_usuario,
        nivel_desc='sector'
    )

    return preds_sector

In [125]:
df['giro'].sample()

5017    Comercio al por mayor de productos farmacéuticos
Name: giro, dtype: object

In [126]:
df['entidad'].sample()

12299    San Luis Potosí
Name: entidad, dtype: object

In [127]:
df[df['año']==2024]

,giro,sector,entidad,año,prima_emitida_neta,prima_retenida,prima_devengada,monto_de_siniestro,gasto_de_ajuste,salvamento,...,monto_de_deducible,monto_coaseguro,n_mero_de_siniestros,recuperacion_de_terceros,recuperacion_de_reaseguro,suma_asegurada,cuota_millar,sin_index,siniestro_neto,net_sin_index
9,Acabado y recubrimiento de textiles,Otros,Guanajuato,2024,883055.23,233712.97,0.0,-1438359.15,1373740.91,0.0,...,0.00,0.0,2.0,0.0,-1314240.26,2556792154,0.345376,0.0,1249622.02,0.0
15,Acabado y recubrimiento de textiles,Otros,Jalisco,2024,657511.67,315597.46,0.0,38000000.00,59576.22,0.0,...,0.00,0.0,1.0,0.0,30655999.92,4307524443,0.152643,0.0,7403576.30,0.0
27,Actividades de seguridad nacional,Gobierno,Ciudad de México,2024,0.00,0.00,0.0,0.00,26190.00,0.0,...,0.00,0.0,1.0,0.0,0.00,242987148,0.000000,0.0,26190.00,0.0
31,Actividades de seguridad nacional,Gobierno,Jalisco,2024,0.00,0.00,0.0,192289.47,43853.95,0.0,...,0.00,0.0,4.0,0.0,192289.47,3034214,0.000000,0.0,43853.95,0.0
73,Administración pública en general,Gobierno,Aguascalientes,2024,2834.51,75.27,0.0,-106076.69,53306.94,0.0,...,850425.08,0.0,14.0,0.0,982373.01,57931665,0.048929,0.0,-1035142.76,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28842,Uniones de crédito e instituciones de ahorro,Otros,Yucatán,2024,31289.42,20525.12,0.0,60000.00,0.00,0.0,...,0.00,0.0,1.0,0.0,21000.00,41726570,0.749868,0.0,39000.00,0.0
28951,Órganos legislativos,Otros,Aguascalientes,2024,168.71,168.71,0.0,-630741.89,4521.20,0.0,...,0.00,0.0,2.0,0.0,-100629.81,2241650,0.075262,0.0,-525590.88,0.0
28960,Órganos legislativos,Otros,Coahuila,2024,1023.99,1023.99,0.0,0.00,4521.20,0.0,...,0.00,0.0,1.0,0.0,0.00,24910000,0.041108,0.0,4521.20,0.0
28967,Órganos legislativos,Otros,Estado de México,2024,46654.07,46653.87,0.0,48030.67,0.00,0.0,...,0.00,0.0,1.0,0.0,39099.86,456812500,0.102130,0.0,8930.81,0.0


In [128]:
df['entidad'].unique()

array(['Baja California', 'Coahuila', 'Colima', 'Ciudad de México',
       'Estado de México', 'Guanajuato', 'Hidalgo', 'Jalisco',
       'Querétaro', 'San Luis Potosí', 'Guerrero', 'Campeche', 'Chiapas',
       'Chihuahua', 'Durango', 'Morelos', 'Nayarit', 'Oaxaca', 'Puebla',
       'Sinaloa', 'Tabasco', 'Tamaulipas', 'Veracruz', 'Yucatán',
       'Zacatecas', 'Sonora', 'Aguascalientes', 'Baja California Sur',
       'Michoacán', 'Nuevo Leon', 'Quintana Roo', 'Tlaxcala',
       'Extranjero'], dtype=object)

In [133]:
giro_usuario='Órganos legislativos'
entidad_usuario='Nuevo Leon'

In [134]:
pred = prediccion_siniestralidad(df_proc, giro_usuario, entidad_usuario)
pred

C:\Users\juanp\anaconda3\envs\DSMod1\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
C:\Users\juanp\anaconda3\envs\DSMod1\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


{2025: 0.0010910305427387357, 2026: 0.0010910305427387357}

In [131]:
df_proc.to_parquet("df_proc.parquet", index=False)

In [132]:
df['entidad'].unique

<bound method Series.unique of 0         Baja California
1                Coahuila
2                  Colima
3        Ciudad de México
4        Ciudad de México
               ...       
28982              Sonora
28983             Tabasco
28984            Veracruz
28985             Yucatán
28986             Yucatán
Name: entidad, Length: 28987, dtype: object>